<a href="https://colab.research.google.com/github/jameschapman19/cca_zoo/blob/main/tutorial_notebooks/cca_zoo_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A tutorial comparing the train and test correlations of different models on MNIST data

In [1]:
# Imports
import numpy as np
from cca_zoo.data import Noisy_MNIST_Dataset, CCA_Dataset
import itertools
import matplotlib.pyplot as plt
from torch.utils.data import Subset, DataLoader
from torch import optim
from cca_zoo.deepmodels import objectives, architectures, CCALightning, DCCA,DCCA_NOI,DVCCA,DCCAE,DTCCA, get_dataloaders
from sklearn.utils.fixes import loguniform
import pytorch_lightning as pl
# Load MNIST Data
N = 500
dataset = Noisy_MNIST_Dataset(mnist_type='FashionMNIST', train=True)
ids = np.arange(min(2 * N, len(dataset)))
np.random.shuffle(ids)
train_ids, val_ids = np.array_split(ids, 2)
val_dataset = Subset(dataset, val_ids)
train_dataset = Subset(dataset, train_ids)
test_dataset = Noisy_MNIST_Dataset(mnist_type='FashionMNIST', train=False)
test_ids = np.arange(min(N, len(test_dataset)))
np.random.shuffle(test_ids)
test_dataset = Subset(test_dataset, test_ids)
(train_view_1, train_view_2),_ = train_dataset.dataset.to_numpy(
    train_dataset.indices)
(val_view_1, val_view_2),_ = val_dataset.dataset.to_numpy(val_dataset.indices)
(test_view_1, test_view_2),_ = test_dataset.dataset.to_numpy(
    test_dataset.indices)
train_loader, val_loader = get_dataloaders(train_dataset, val_dataset)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))
# Settings

# The number of latent dimensions across models
latent_dims = 2
# The number of cv used for cross-validation/hyperparameter tuning
cv = 3
# Number of iterations for iterative algorithms
max_iter = 2
# number of epochs for deep models
epochs = 20

# Canonical Correlation Analysis

In [2]:
from cca_zoo.models import CCA, CCA_ALS
"""
### Linear CCA by eigendecomposition
"""
linear_cca = CCA(latent_dims=latent_dims)

linear_cca.fit((train_view_1, train_view_2))

linear_cca_results = np.stack(
    (linear_cca.score((train_view_1, train_view_2)), linear_cca.score((test_view_1, test_view_2))))

"""
### Linear CCA by alternating least squares (can pass more than 2 views)
"""

linear_cca_als = CCA_ALS(latent_dims=latent_dims)

linear_cca_als.fit((train_view_1, train_view_2))

linear_cca_als_results = np.stack(
    (linear_cca_als.score((train_view_1, train_view_2)), linear_cca_als.score((test_view_1, test_view_2))))

# Partial Least Squares


In [3]:
from cca_zoo.models import PLS, PLS_ALS
"""
### PLS (2 views)
"""
pls = PLS(latent_dims=latent_dims)

pls.fit((train_view_1, train_view_2))

pls_results = np.stack(
    (pls.score((train_view_1, train_view_2)), pls.score((test_view_1, test_view_2))))

pls_als = PLS_ALS(latent_dims=latent_dims)

pls_als.fit((train_view_1, train_view_2))

pls_als_results = np.stack(
    (pls_als.score((train_view_1, train_view_2)), pls_als.score((test_view_1, test_view_2))))

# Extension to multiple views



In [4]:
from cca_zoo.models import GCCA, MCCA, PLS_ALS
"""
### (Regularized) Generalized CCA(can pass more than 2 views)
"""
train_view_3=train_view_1+np.random.rand(*train_view_1.shape)
test_view_3=test_view_1+np.random.rand(*test_view_1.shape)

# small ammount of regularisation added since data is not full rank
c=[0.5,0.5,0.5]

gcca = GCCA(latent_dims=latent_dims,c=c)

gcca.fit((train_view_1, train_view_2,train_view_3))

gcca_results = np.stack((gcca.score((train_view_1, train_view_2, train_view_3)), gcca.score((test_view_1, test_view_2, test_view_3))))

"""
### (Regularized) Multiset CCA(can pass more than 2 views)
"""

mcca = MCCA(latent_dims=latent_dims, c=c)

mcca.fit((train_view_1, train_view_2,train_view_1))

mcca_results = np.stack((mcca.score((train_view_1, train_view_2, train_view_3)), mcca.score((test_view_1, test_view_2, test_view_3))))

"""
### Multiset CCA by alternating least squares
"""
mcca_als = CCA_ALS(latent_dims=latent_dims, max_iter=max_iter)

mcca_als.fit((train_view_1, train_view_2,train_view_3))

mcca_als_results = np.stack(
    (mcca_als.score((train_view_1, train_view_2, train_view_3)), mcca_als.score((test_view_1, test_view_2, test_view_3))))

"""
### Multiset PLS by alternating least squares
"""
mcca_pls = PLS_ALS(latent_dims=latent_dims)

mcca_pls.fit((train_view_1, train_view_2,train_view_1))

mcca_pls_results = np.stack(
    (mcca_als.score((train_view_1, train_view_2, train_view_3)), mcca_pls.score((test_view_1, test_view_2, test_view_3))))

C:\Users\chapm\PycharmProjects\cca_zoo\cca_zoo\models\innerloop.py:84: UserWarning: For more than 2 views require generalized=True
  warnings.warn("For more than 2 views require generalized=True")
C:\Users\chapm\PycharmProjects\cca_zoo\cca_zoo\models\innerloop.py:84: UserWarning: For more than 2 views require generalized=True
  warnings.warn("For more than 2 views require generalized=True")
C:\Users\chapm\PycharmProjects\cca_zoo\cca_zoo\models\innerloop.py:84: UserWarning: For more than 2 views require generalized=True
  warnings.warn("For more than 2 views require generalized=True")


# Tensor CCA

In [5]:
from cca_zoo.models import TCCA
"""
### (Regularized) Tensor CCA(can pass more than 2 views)
"""

tcca = TCCA(latent_dims=latent_dims, c=c)

#memory requirement for tensor is massive so take first 100 features
tcca.fit((train_view_1[:,:100], train_view_2[:,:100],train_view_3[:,:100]))

tcca_results = np.stack((tcca.score((train_view_1[:,:100], train_view_2[:,:100], train_view_3[:,:100])), tcca.score((test_view_1[:,:100], test_view_2[:,:100], test_view_3[:,:100]))))

reconstruction error=0.9853859740905844
iteration 1, reconstruction error: 0.9618427894635867, decrease = 0.02354318462699767, unnormalized = 24.624750217020292
iteration 2, reconstruction error: 0.9617552471717098, decrease = 8.754229187690754e-05, unnormalized = 24.62250899101693
iteration 3, reconstruction error: 0.9617551494921354, decrease = 9.767957442896602e-08, unnormalized = 24.622506490259894
iteration 4, reconstruction error: 0.9617551494358002, decrease = 5.633515876013462e-11, unnormalized = 24.62250648881762
PARAFAC converged after 4 iterations


# Weighted GCCA/Missing Observation GCCA

In [6]:
#observation_matrix
K = np.ones((3, N))
K[0, 200:] = 0
K[1, :100] = 0

#view weights
view_weights=[1,2,1.2]

c=[0.5,0.5,0.5]

gcca = GCCA(latent_dims=latent_dims,c=c,view_weights=view_weights)

gcca.fit((train_view_1, train_view_2,train_view_1),K=K)

gcca_results = np.stack((gcca.score((train_view_1, train_view_2)), gcca.score((test_view_1, test_view_2))))

# Regularised CCA solutions based on alternating minimisation/alternating least squares

We implement Witten's penalized matrix decomposition form of sparse CCA using 'pmd'

We implement Waaijenborg's penalized CCA using elastic net using 'elastic'

We implement Mai's sparse CCA using 'scca'

Furthermore, any of these methods can be extended to multiple views. Witten describes this method explicitly.

In [7]:
from cca_zoo.model_selection import GridSearchCV, RandomizedSearchCV
from cca_zoo.models import rCCA, PMD,SCCA,ElasticCCA

def scorer(estimator,X):
  dim_corrs=estimator.score(X)
  return dim_corrs.mean()

"""
### Ridge CCA (can pass more than 2 views)
"""
c1 = [0.1, 0.3, 0.7, 0.9]
c2 = [0.1, 0.3, 0.7, 0.9]
param_grid = {'c': [c1,c2]}

ridge = GridSearchCV(rCCA(latent_dims=latent_dims),param_grid=param_grid,
    cv=cv,
    verbose=True,scoring=scorer).fit([train_view_1,train_view_2]).best_estimator_

ridge_results = np.stack((ridge.score((train_view_1,train_view_2)), ridge.score((test_view_1, test_view_2))))

"""
### Sparse CCA (Penalized Matrix Decomposition) (can pass more than 2 views)
"""

# PMD
c1 = [1, 3, 7, 9]
c2 = [1, 3, 7, 9]
param_grid = {'c': [c1,c2]}

pmd = GridSearchCV(PMD(latent_dims=latent_dims),param_grid=param_grid,
    cv=cv,
    verbose=True,scoring=scorer).fit([train_view_1,train_view_2]).best_estimator_

pmd_results = np.stack((pmd.score((train_view_1,train_view_2)), pmd.score((test_view_1, test_view_2))))

"""
### Sparse CCA (can pass more than 2 views)
"""

# Sparse CCA
c1 = [0.00001, 0.0001]
c2 = [0.00001, 0.0001]
param_grid = {'c': [c1,c2]}

scca = GridSearchCV(SCCA(latent_dims=latent_dims),param_grid=param_grid,
    cv=cv,
    verbose=True,scoring=scorer).fit([train_view_1,train_view_2]).best_estimator_

scca_results = np.stack(
    (scca.score((train_view_1,train_view_2)), scca.score((test_view_1, test_view_2))))


"""
### Elastic CCA (can pass more than 2 views)
"""

# Elastic CCA
c1 = loguniform(1e-4, 1e0)
c2 = loguniform(1e-4, 1e0)
l1_1 = loguniform(1e-4, 1e0)
l1_2 = loguniform(1e-4, 1e0)
param_grid = {'c': [c1,c2], 'l1_ratio': [l1_1,l1_2]}

elastic = RandomizedSearchCV(ElasticCCA(latent_dims=latent_dims),param_distributions=param_grid,
    cv=cv,
    verbose=True,n_iter=5,scoring=scorer).fit([train_view_1,train_view_2]).best_estimator_

elastic_results = np.stack(
    (elastic.score((train_view_1,train_view_2)), elastic.score((test_view_1, test_view_2))))

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Fitting 3 folds for each of 16 candidates, totalling 48 fits
Fitting 3 folds for each of 4 candidates, totalling 12 fits
Fitting 3 folds for each of 5 candidates, totalling 15 fits


# Kernel CCA

In [ ]:
from cca_zoo.models import KCCA
"""
### Kernel CCA

Similarly, we can use kernel CCA methods with [method='kernel']

We can use different kernels and their associated parameters in a similar manner to before
- regularized linear kernel CCA: parameters :  'kernel'='linear', 0<'c'<1
- polynomial kernel CCA: parameters : 'kernel'='poly', 'degree', 0<'c'<1
- gaussian rbf kernel CCA: parameters : 'kernel'='gaussian', 'sigma', 0<'c'<1
"""
# %%
# r-kernel cca
c1 = [0.9, 0.99]
c2 = [0.9, 0.99]

param_grid = {'kernel': ['linear'], 'c': [c1,c2]}

kernel_reg = GridSearchCV(KCCA(latent_dims=latent_dims),param_grid=param_grid,
    cv=cv,
    verbose=True,scoring=scorer).fit([train_view_1,train_view_2]).best_estimator_
kernel_reg_results = np.stack((
    kernel_reg.score((train_view_1,train_view_2)),
    kernel_reg.score((test_view_1, test_view_2))))

# kernel cca (poly)
degree1 = [2, 3]
degree2 = [2, 3]

param_grid = {'kernel': ['poly'], 'degree': [degree1,degree2],
                    'c': [c1,c2]}

kernel_poly = GridSearchCV(KCCA(latent_dims=latent_dims),param_grid=param_grid,
    cv=cv,
    verbose=True,scoring=scorer).fit([train_view_1,train_view_2]).best_estimator_

kernel_poly_results = np.stack((
    kernel_poly.score((train_view_1,train_view_2)),
    kernel_poly.score((test_view_1, test_view_2))))

# kernel cca (gaussian)
gamma1 = [1e+1, 1e+2, 1e+3]
gamma2 = [1e+1, 1e+2, 1e+3]

param_grid = {'kernel': ['rbf'], 'gamma': [gamma1,gamma2],
                    'c': [c1,c2]}

kernel_gaussian = GridSearchCV(KCCA(latent_dims=latent_dims),param_grid=param_grid,
    cv=cv,
    verbose=True,scoring=scorer).fit([train_view_1,train_view_2]).best_estimator_

kernel_gaussian_results = np.stack((
    kernel_gaussian.score((train_view_1,train_view_2)),
    kernel_gaussian.score((test_view_1, test_view_2))))

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Fitting 3 folds for each of 16 candidates, totalling 48 fits
Fitting 3 folds for each of 36 candidates, totalling 108 fits


c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value 

c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value 

c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value 

# Deep CCA

DCCA can be optimized using Andrew's original tracenorm objective or Wang's DCCA by nonlinear orthogonal iterations using the argument als=True.

In [6]:
"""
### Deep Learning

We also have deep CCA methods (and autoencoder variants)
- Deep CCA (DCCA)
- Deep Canonically Correlated Autoencoders (DCCAE)

"""

# %%
# DCCA
print('DCCA')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
dcca_model = DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2])

dcca_model = CCALightning(dcca_model)
trainer = pl.Trainer(max_epochs=epochs, progress_bar_refresh_rate=1, log_every_n_steps=1, logger=False)
trainer.fit(dcca_model, train_loader, val_loader)

dcca_results = np.stack((dcca_model.score(train_loader), dcca_model.score(test_loader)))

# DCCA_NOI
print('DCCA by non-linear orthogonal iterations')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
dcca_noi_model = DCCA_NOI(latent_dims=latent_dims, encoders=[encoder_1, encoder_2],N=len(train_dataset))

dcca_noi_model = CCALightning(dcca_noi_model)
trainer = pl.Trainer(max_epochs=epochs, progress_bar_refresh_rate=1, log_every_n_steps=1, logger=False)
trainer.fit(dcca_noi_model, train_loader, val_loader)

dcca_noi_results = np.stack((dcca_noi_model.score(train_loader), dcca_noi_model.score(test_loader)))

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type | Params
-------------------------------
0 | model | DCCA | 201 K 
-------------------------------
201 K     Trainable params
0         Non-trainable params
201 K     Total params
0.806     Total estimated model params size (MB)


DCCA


Validation sanity check: 0it [00:00, ?it/s]

c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# DCCA with custom optimizers and schedulers

In [7]:
# DCCA
print('DCCA')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
dcca_model = DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2])
optimizer = optim.Adam(dcca_model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 1)
dcca_model = CCALightning(dcca_model)
trainer = pl.Trainer(max_epochs=epochs, progress_bar_refresh_rate=1, log_every_n_steps=1, logger=False)
trainer.fit(dcca_model, train_loader, val_loader)

dcca_results = np.stack((dcca_model.score(train_loader), dcca_model.score(test_loader)))

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\pytorch_lightning\trainer\trainer.py:1303: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

  | Name  | Type | Params
-------------------------------
0 | model | DCCA | 201 K 
-------------------------------
201 K     Trainable params
0         Non-trainable params
201 K     Total params
0.806     Total estimated model params size (MB)


DCCA


Validation sanity check: 0it [00:00, ?it/s]

c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# DGCCA and DMCCA for more than 2 views

The only change we need to make is to the objective argument to perform DGCCA and DMCCA.

In [8]:
# DGCCA
print('DGCCA')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
dgcca_model = DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], objective=objectives.GCCA)

dgcca_model = CCALightning(dgcca_model)
trainer = pl.Trainer(max_epochs=epochs, progress_bar_refresh_rate=1, log_every_n_steps=1, logger=False)
trainer.fit(dgcca_model, train_loader, val_loader)

dgcca_results = np.stack((dgcca_model.score(train_loader), dgcca_model.score(test_loader)))

# DMCCA
print('DMCCA')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
dmcca_model = DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], objective=objectives.MCCA)

dmcca_model = CCALightning(dmcca_model)
trainer = pl.Trainer(max_epochs=epochs, progress_bar_refresh_rate=1, log_every_n_steps=1, logger=False)
trainer.fit(dmcca_model, train_loader, val_loader)

dmcca_results = np.stack((dmcca_model.score(train_loader), dmcca_model.score(test_loader)))

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\pytorch_lightning\trainer\trainer.py:1303: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

  | Name  | Type | Params
-------------------------------
0 | model | DCCA | 201 K 
-------------------------------
201 K     Trainable params
0         Non-trainable params
201 K     Total params
0.806     Total estimated model params size (MB)


DGCCA


Validation sanity check: 0it [00:00, ?it/s]

c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Deep Canonically Correlated Autoencoders
We need to add decoders in order to model deep canonically correlated autoencoders and we also use the DCCAE class which inherits from DCCA

In [9]:
# DCCAE
print('DCCAE')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
decoder_1 = architectures.Decoder(latent_dims=latent_dims, feature_size=784)
decoder_2 = architectures.Decoder(latent_dims=latent_dims, feature_size=784)
dccae_model = DCCAE(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], decoders=[decoder_1, decoder_2])

dccae_model = CCALightning(dccae_model)
trainer = pl.Trainer(max_epochs=epochs, progress_bar_refresh_rate=1, log_every_n_steps=1, logger=False)
trainer.fit(dccae_model, train_loader, val_loader)

dccae_results = np.stack((dccae_model.score(train_loader), dccae_model.score(test_loader)))

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\pytorch_lightning\trainer\trainer.py:1303: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

  | Name  | Type  | Params
--------------------------------
0 | model | DCCAE | 404 K 
--------------------------------
404 K     Trainable params
0         Non-trainable params
404 K     Total params
1.618     Total estimated model params size (MB)


DCCAE


Validation sanity check: 0it [00:00, ?it/s]

c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Deep Variational CCA

In [10]:
"""
### Deep Variational Learning
Finally we have Deep Variational CCA methods.
- Deep Variational CCA (DVCCA)
- Deep Variational CCA - private (DVVCA_p)

These are both implemented by the DVCCA class with private=True/False and both_encoders=True/False. If both_encoders,
the encoder to the shared information Q(z_shared|x) is modelled for both x_1 and x_2 whereas if both_encoders is false
it is modelled for x_1 as in the paper
"""

# %%
# DVCCA (technically bi-DVCCA)
print('DVCCA')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
decoder_1 = architectures.Decoder(latent_dims=latent_dims, feature_size=784, norm_output=True)
decoder_2 = architectures.Decoder(latent_dims=latent_dims, feature_size=784, norm_output=True)
dvcca_model = DVCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], decoders=[decoder_1, decoder_2])

dvcca_model = CCALightning(dvcca_model)
trainer = pl.Trainer(max_epochs=epochs, progress_bar_refresh_rate=1, log_every_n_steps=1, logger=False)
trainer.fit(dvcca_model, train_loader, val_loader)

dvcca_model_results = np.stack((dvcca_model.score(train_loader), dvcca_model.score(test_loader)))

# DVCCA_private (technically bi-DVCCA_private)
print('DVCCA_private')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
private_encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
private_encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784, variational=True)
decoder_1 = architectures.Decoder(latent_dims=latent_dims * 2, feature_size=784, norm_output=True)
decoder_2 = architectures.Decoder(latent_dims=latent_dims * 2, feature_size=784, norm_output=True)
dvccap_model = DVCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2], decoders=[decoder_1, decoder_2],
                           private_encoders=[private_encoder_1, private_encoder_2])

dvccap_model = CCALightning(dvccap_model)
trainer = pl.Trainer(max_epochs=epochs, progress_bar_refresh_rate=1, log_every_n_steps=1, logger=False)
trainer.fit(dvccap_model, train_loader, val_loader)

dvccap_model_results = np.stack((dvccap_model.score(train_loader), dvccap_model.score(test_loader)))

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type  | Params
--------------------------------
0 | model | DVCCA | 405 K 
--------------------------------
405 K     Trainable params
0         Non-trainable params
405 K     Total params
1.620     Total estimated model params size (MB)


DVCCA


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Convolutional Deep CCA (and using other architectures)
We provide a standard CNN encoder and decoder but users can build their own encoders and decoders by inheriting BaseEncoder and BaseDecoder for seamless integration with the pipeline

In [15]:
print('Convolutional DCCA')
encoder_1 = architectures.CNNEncoder(latent_dims=latent_dims, channels=[3, 3])
encoder_2 = architectures.CNNEncoder(latent_dims=latent_dims, channels=[3, 3])
dcca_conv_model = DCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2])

conv_train_view_1=train_view_1.reshape((-1, 1, 28, 28))
conv_train_view_2=train_view_2.reshape((-1, 1, 28, 28))
conv_test_view_1=test_view_1.reshape((-1, 1, 28, 28))
conv_test_view_2=test_view_2.reshape((-1, 1, 28, 28))
conv_dataset=CCA_Dataset((conv_train_view_1,conv_train_view_2))
test_conv_dataset=CCA_Dataset((conv_test_view_1,conv_test_view_2))
conv_train_loader=get_dataloaders(conv_dataset)
conv_test_loader=get_dataloaders(test_conv_dataset)

dcca_conv_model = CCALightning(dcca_conv_model)
trainer = pl.Trainer(max_epochs=epochs, progress_bar_refresh_rate=1, log_every_n_steps=1, logger=False)
trainer.fit(dcca_conv_model, conv_train_loader)

dcca_conv_results = np.stack((
    dcca_conv_model.score(conv_train_loader),
    dcca_conv_model.score(conv_test_loader)))

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type | Params
-------------------------------
0 | model | DCCA | 9.6 K 
-------------------------------
9.6 K     Trainable params
0         Non-trainable params
9.6 K     Total params
0.038     Total estimated model params size (MB)


Convolutional DCCA


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

# DTCCA

In [17]:
# %%
# DTCCA
print('DTCCA')
encoder_1 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
encoder_2 = architectures.Encoder(latent_dims=latent_dims, feature_size=784)
dtcca_model = DTCCA(latent_dims=latent_dims, encoders=[encoder_1, encoder_2])

dtcca_model = CCALightning(dtcca_model)
trainer = pl.Trainer(max_epochs=epochs, progress_bar_refresh_rate=1, log_every_n_steps=1, logger=False)
trainer.fit(dtcca_model, train_loader, val_loader)

dtcca_results = np.stack((dtcca_model.score(train_loader), dtcca_model.score(test_loader)))

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\pytorch_lightning\trainer\trainer.py:1303: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

  | Name  | Type  | Params
--------------------------------
0 | model | DTCCA | 201 K 
--------------------------------
201 K     Trainable params
0         Non-trainable params
201 K     Total params
0.806     Total estimated model params size (MB)


DTCCA


Validation sanity check: 0it [00:00, ?it/s]

c:\users\chapm\pycharmprojects\cca_zoo\venv\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


reconstruction error=0.0
iteration 1, reconstruction error: 0.0, decrease = 0.0, unnormalized = 0.0
PARAFAC converged after 1 iterations


...

reconstruction error=1.6417717997295322e-08
iteration 1, reconstruction error: 0.0, decrease = 1.6417717997295322e-08, unnormalized = 0.0
iteration 2, reconstruction error: 0.0, decrease = 0.0, unnormalized = 0.0
PARAFAC converged after 2 iterations
reconstruction error=0.0
iteration 1, reconstruction error: 1.6105270747910115e-08, decrease = -1.6105270747910115e-08, unnormalized = 1.0536712127723509e-08
iteration 2, reconstruction error: 1.6105270747910115e-08, decrease = 0.0, unnormalized = 1.0536712127723509e-08
PARAFAC converged after 2 iterations


Validating: 0it [00:00, ?it/s]

reconstruction error=0.0
iteration 1, reconstruction error: 1.591287807057014e-08, decrease = -1.591287807057014e-08, unnormalized = 1.0536712127723509e-08
iteration 2, reconstruction error: 0.0, decrease = 1.591287807057014e-08, unnormalized = 0.0
iteration 3, reconstruction error: 1.591287807057014e-08, decrease = -1.591287807057014e-08, unnormalized = 1.0536712127723509e-08
iteration 4, reconstruction error: 1.591287807057014e-08, decrease = 0.0, unnormalized = 1.0536712127723509e-08
PARAFAC converged after 4 iterations
reconstruction error=1.6168068144444113e-08
iteration 1, reconstruction error: 0.0, decrease = 1.6168068144444113e-08, unnormalized = 0.0
iteration 2, reconstruction error: 1.6168068144444113e-08, decrease = -1.6168068144444113e-08, unnormalized = 1.0536712127723509e-08
iteration 3, reconstruction error: 1.6168068144444113e-08, decrease = 0.0, unnormalized = 1.0536712127723509e-08
PARAFAC converged after 3 iterations


Validating: 0it [00:00, ?it/s]

reconstruction error=2.7153777508762245e-08
iteration 1, reconstruction error: 1.567724075419909e-08, decrease = 1.1476536754563153e-08, unnormalized = 1.0536712127723509e-08
iteration 2, reconstruction error: 2.2170966495176562e-08, decrease = -6.493725740977471e-09, unnormalized = 1.4901161193847656e-08
PARAFAC converged after 2 iterations
reconstruction error=0.0
iteration 1, reconstruction error: 0.0, decrease = 0.0, unnormalized = 0.0
PARAFAC converged after 1 iterations
